# Estimating local and global feature importance scores using DiCE

Summaries of counterfactual examples can be used to estimate importance of features. Intuitively, a feature that is changed more often to generate a proximal counterfactual is an important feature. We use this intuition to build a feature importance score. 

This score can be interpreted as a measure of the **necessity** of a feature to cause a particular model output. That is, if the feature's value changes, then it is likely that the model's output class will also change (or the model's output will significantly change in case of regression model).  

Below we show how counterfactuals can be used to provide local feature importance scores for any input, and how those scores can be combined to yield a global importance score for each feature.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

## Preliminaries: Loading the data and ML model

In [3]:
from dice_ml.utils import helpers # helper functions
dataset = helpers.load_adult_income_dataset()
helpers.get_adult_data_info()

{'age': 'age',
 'workclass': 'type of industry (Government, Other/Unknown, Private, Self-Employed)',
 'education': 'education level (Assoc, Bachelors, Doctorate, HS-grad, Masters, Prof-school, School, Some-college)',
 'marital_status': 'marital status (Divorced, Married, Separated, Single, Widowed)',
 'occupation': 'occupation (Blue-Collar, Other/Unknown, Professional, Sales, Service, White-Collar)',
 'race': 'white or other race?',
 'gender': 'male or female?',
 'hours_per_week': 'total work hours per week',
 'income': '0 (<=50K) vs 1 (>50K)'}

In [4]:
target = dataset["income"]
# Split data into train and test
from sklearn.model_selection import train_test_split
datasetX = dataset.drop("income", axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX, 
                                                    target, 
                                                    test_size = 0.2,
                                                    random_state=0,
                                                    stratify=target)

numerical=["age", "hours_per_week"]
categorical = x_train.columns.difference(numerical) 
from sklearn.compose import ColumnTransformer

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', RandomForestClassifier())])
model = clf.fit(x_train, y_train)

In [5]:
import dice_ml
from dice_ml import Dice

d = dice_ml.Data(dataframe=dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')
m = dice_ml.Model(model=model, backend="sklearn")

## Local feature importance

We first generate counterfactuals for a given input point. 

In [6]:
exp = Dice(d, m, method="random")
query_instance = x_train[2:3]
e1 = exp.generate_counterfactuals(query_instance, total_CFs=2)
e1.visualize_as_dataframe(show_only_changes=True)

Diverse Counterfactuals found! total time taken: 00 min 00 sec
Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,31,Private,Some-college,Divorced,Blue-Collar,White,Male,40,0.0



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,62.0,Government,Prof-school,Widowed,Other/Unknown,Other,-,46.0,0.51
1,43.0,Self-Employed,-,Married,Professional,-,Female,31.0,0.64


These can now be used to calculate the feature importance scores. 

In [7]:
imp = exp.feature_importance(query_instance, cf_examples_list=e1.cf_examples_list)
print(imp.local_importance)

[{'workclass': 1.0, 'education': 0.5, 'marital_status': 1.0, 'occupation': 1.0, 'race': 0.5, 'gender': 0.5, 'age': 1.0, 'hours_per_week': 1.0}]


Feature importance can also be estimated directly, by leaving the `cf_examples_list` argument blank.

In [8]:
imp = exp.feature_importance(query_instance, posthoc_sparsity_param=None)
print(imp.local_importance)

Diverse Counterfactuals found! total time taken: 00 min 00 sec
[{'workclass': 0.5, 'education': 0.8, 'marital_status': 1.0, 'occupation': 1.0, 'race': 0.3, 'gender': 0.7, 'age': 1.0, 'hours_per_week': 1.0}]


## Global importance

For global importance, we need to generate counterfactuals for a representative sample of the dataset. 

In [9]:
cobj=exp.feature_importance(x_train[0:10], total_CFs=2, posthoc_sparsity_param=None)
print(cobj.summary_importance)

Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
{'workclass': 0.95, 'education': 0.9, 'marital_status': 0.8, 'occupation': 0.95, 'race': 0.45, 'gender': 0.35, 'age': 1.0, 'hours_per_week': 1.0}


## Convert the counterfactual output to json

In [10]:
json_str = cobj.to_json()
print(json_str)

{
  "cf_examples_list": [
    "{\"data_interface\": {\"outcome_name\": \"income\", \"data_df\": \"dummy_data\"}, \"model_type\": \"classifier\", \"desired_class\": 1.0, \"desired_range\": null, \"test_instance_df\": \"{\\\"age\\\":{\\\"0\\\":38},\\\"workclass\\\":{\\\"0\\\":\\\"Private\\\"},\\\"education\\\":{\\\"0\\\":\\\"HS-grad\\\"},\\\"marital_status\\\":{\\\"0\\\":\\\"Married\\\"},\\\"occupation\\\":{\\\"0\\\":\\\"Blue-Collar\\\"},\\\"race\\\":{\\\"0\\\":\\\"White\\\"},\\\"gender\\\":{\\\"0\\\":\\\"Male\\\"},\\\"hours_per_week\\\":{\\\"0\\\":44},\\\"income\\\":{\\\"0\\\":0.076}}\", \"final_cfs_df\": \"{\\\"age\\\":{\\\"0\\\":62.0,\\\"1\\\":43.0},\\\"workclass\\\":{\\\"0\\\":\\\"Government\\\",\\\"1\\\":\\\"Self-Employed\\\"},\\\"education\\\":{\\\"0\\\":\\\"Prof-school\\\",\\\"1\\\":\\\"Some-college\\\"},\\\"marital_status\\\":{\\\"0\\\":\\\"Widowed\\\",\\\"1\\\":\\\"Married\\\"},\\\"occupation\\\":{\\\"0\\\":\\\"Other\\\\/Unknown\\\",\\\"1\\\":\\\"Professional\\\"},\\\"race\\\":{

## Convert the json output to a counterfactual object

In [11]:
imp_r=imp.from_json(json_str)
print([o.visualize_as_dataframe() for o in imp_r.cf_examples_list])
print(imp_r.local_importance)
print(imp_r.summary_importance)

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,38,Private,HS-grad,Married,Blue-Collar,White,Male,44,0.076



Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,62,Government,Prof-school,Widowed,Other/Unknown,Other,Male,46,0.51
1,43,Self-Employed,Some-college,Married,Professional,White,Female,31,0.64


Query instance (original outcome : 1)


...

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,32,Private,HS-grad,Married,Blue-Collar,White,Male,40,0.277



Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,62,Government,Prof-school,Widowed,Other/Unknown,Other,Male,46,0.51
1,43,Self-Employed,Some-college,Married,Professional,White,Female,31,0.64


[None, None, None, None, None, None, None, None, None, None]
[{'workclass': 1.0, 'education': 1.0, 'marital_status': 0.5, 'occupation': 1.0, 'race': 0.5, 'gender': 0.5, 'age': 1.0, 'hours_per_week': 1.0}, {'workclass': 0.5, 'education': 1.0, 'marital_status': 1.0, 'occupation': 1.0, 'race': 0.0, 'gender': 0.0, 'age': 1.0, 'hours_per_week': 1.0}, {'workclass': 1.0, 'education': 0.5, 'marital_status': 1.0, 'occupation': 1.0, 'race': 0.5, 'gender': 0.5, 'age': 1.0, 'hours_per_week': 1.0}, {'workclass': 1.0, 'education': 0.5, 'marital_status': 0.5, 'occupation': 1.0, 'race': 0.5, 'gender': 0.5, 'age': 1.0, 'hours_per_week': 1.0}, {'workclass': 1.0, 'education': 1.0, 'marital_status': 1.0, 'occupation': 1.0, 'race': 0.5, 'gender': 0.5, 'age': 1.0, 'hours_per_week': 1.0}, {'workclass': 1.0, 'education': 1.0, 'marital_status': 1.0, 'occupation': 0.5, 'race': 0.0, 'gender': 0.0, 'age': 1.0, 'hours_per_week': 1.0}, {'workclass': 1.0, 'education': 1.0, 'marital_status': 0.5, 'occupation': 1.0, '